In [1]:
"""
20190125

Conditional GAN

@author: https://github.com/eriklindernoren/Keras-GAN/blob/master/cgan/cgan.py
"""

'\n20190125\n\nConditional GAN\n\n@author: https://github.com/eriklindernoren/Keras-GAN/blob/master/cgan/cgan.py\n'

# Import

In [2]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
# config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.3
set_session(tf.Session(config=config))

import matplotlib.pyplot as plt

import numpy as np

Using TensorFlow backend.


# Conditional GAN

In [4]:
class CGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.num_classes = 10
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss=['binary_crossentropy'],
                                   optimizer=optimizer,
                                   metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise and the target label as input
        # and generates the corresponding digit of that label
        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,))
        img = self.generator([noise, label])

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated image as input and determines validity
        # and the label of that image
        valid = self.discriminator([img, label])

        # The combined model  (stacked generator and discriminator)
        # Trains generator to fool discriminator
        self.combined = Model([noise, label], valid)
        self.combined.compile(loss=['binary_crossentropy'],
                              optimizer=optimizer)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,), dtype='int32')
        # Embed the num_classes as vectors and define the output shape as same as image shape
        # https://keras.io/layers/embeddings/
        label_embedding = Flatten()(Embedding(self.num_classes, self.latent_dim)(label))

        model_input = multiply([noise, label_embedding])
        img = model(model_input)

        return Model([noise, label], img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Dense(512, input_dim=np.prod(self.img_shape)))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.4))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.4))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=self.img_shape)
        label = Input(shape=(1,), dtype='int32')
        label_embedding = Flatten()(Embedding(self.num_classes, np.prod(self.img_shape))(label))
        flat_img = Flatten()(img)

        # img's dim
        model_input = multiply([flat_img, label_embedding])

        validity = model(model_input)

        return Model([img, label], validity)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, y_train), (_, _) = mnist.load_data()

        # Configure input
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)
        y_train = y_train.reshape(-1, 1)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs, labels = X_train[idx], y_train[idx]

            # Sample noise as generator input
            noise = np.random.normal(0, 1, (batch_size, 100))

            # Generate a half batch of new images
            gen_imgs = self.generator.predict([noise, labels])

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(
                [imgs, labels], valid)
            d_loss_fake = self.discriminator.train_on_batch(
                [gen_imgs, labels], fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Condition on labels
            sampled_labels = np.random.randint(
                0, 10, batch_size).reshape(-1, 1)

            # Train the generator
            g_loss = self.combined.train_on_batch(
                [noise, sampled_labels], valid)

            # Plot the progress
            print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" %
                  (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.sample_images(epoch)

    def sample_images(self, epoch):
        r, c = 2, 5  # row and column
        # same as GAN, input noise
        noise = np.random.normal(0, 1, (r * c, 100))
        sampled_labels = np.arange(0, 10).reshape(-1, 1)  # sample label s

        gen_imgs = self.generator.predict([noise, sampled_labels])

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i, j].imshow(gen_imgs[cnt, :, :, 0], cmap='gray')
                axs[i, j].set_title("Digit: %d" % sampled_labels[cnt])
                axs[i, j].axis('off')
                cnt += 1
        fig.savefig("CGAN_Images/%d.png" % epoch)
        plt.close()


if __name__ == '__main__':
    cgan = CGAN()
    cgan.train(epochs=20000, batch_size=32, sample_interval=200)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 512)               262656    
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 512)               262656    
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 512)               0         
__________

91 [D loss: 0.083186, acc.: 98.44%] [G loss: 6.972547]
92 [D loss: 0.279703, acc.: 84.38%] [G loss: 7.050364]
93 [D loss: 0.271561, acc.: 89.06%] [G loss: 6.600245]
94 [D loss: 0.381388, acc.: 79.69%] [G loss: 7.037290]
95 [D loss: 0.571680, acc.: 71.88%] [G loss: 6.490625]
96 [D loss: 0.190152, acc.: 92.19%] [G loss: 6.230112]
97 [D loss: 0.453477, acc.: 81.25%] [G loss: 6.005981]
98 [D loss: 0.329222, acc.: 84.38%] [G loss: 6.635345]
99 [D loss: 0.422875, acc.: 87.50%] [G loss: 6.180662]
100 [D loss: 0.486317, acc.: 71.88%] [G loss: 6.801296]
101 [D loss: 0.390499, acc.: 82.81%] [G loss: 6.308372]
102 [D loss: 0.422710, acc.: 78.12%] [G loss: 6.547946]
103 [D loss: 0.386651, acc.: 79.69%] [G loss: 6.829842]
104 [D loss: 0.334514, acc.: 82.81%] [G loss: 6.853463]
105 [D loss: 0.311335, acc.: 82.81%] [G loss: 6.879696]
106 [D loss: 0.378107, acc.: 85.94%] [G loss: 5.805555]
107 [D loss: 0.506443, acc.: 67.19%] [G loss: 6.432735]
108 [D loss: 0.459680, acc.: 84.38%] [G loss: 5.687568]
1

243 [D loss: 0.392330, acc.: 81.25%] [G loss: 4.334571]
244 [D loss: 0.423877, acc.: 79.69%] [G loss: 4.208665]
245 [D loss: 0.501658, acc.: 79.69%] [G loss: 3.594912]
246 [D loss: 0.408209, acc.: 85.94%] [G loss: 4.297361]
247 [D loss: 0.307419, acc.: 85.94%] [G loss: 4.046293]
248 [D loss: 0.377148, acc.: 79.69%] [G loss: 4.129125]
249 [D loss: 0.543841, acc.: 71.88%] [G loss: 3.548437]
250 [D loss: 0.333605, acc.: 87.50%] [G loss: 3.659060]
251 [D loss: 0.483530, acc.: 82.81%] [G loss: 3.317986]
252 [D loss: 0.572202, acc.: 73.44%] [G loss: 3.659546]
253 [D loss: 0.412210, acc.: 82.81%] [G loss: 3.866033]
254 [D loss: 0.512404, acc.: 79.69%] [G loss: 3.626776]
255 [D loss: 0.483992, acc.: 75.00%] [G loss: 4.707797]
256 [D loss: 0.405479, acc.: 78.12%] [G loss: 4.095501]
257 [D loss: 0.468187, acc.: 81.25%] [G loss: 3.549708]
258 [D loss: 0.638798, acc.: 68.75%] [G loss: 3.402160]
259 [D loss: 0.556937, acc.: 71.88%] [G loss: 3.181000]
260 [D loss: 0.440495, acc.: 81.25%] [G loss: 3.

393 [D loss: 0.646125, acc.: 65.62%] [G loss: 1.610093]
394 [D loss: 0.477497, acc.: 81.25%] [G loss: 1.763635]
395 [D loss: 0.549290, acc.: 67.19%] [G loss: 2.085583]
396 [D loss: 0.505228, acc.: 78.12%] [G loss: 1.816428]
397 [D loss: 0.466563, acc.: 73.44%] [G loss: 1.858398]
398 [D loss: 0.564893, acc.: 65.62%] [G loss: 1.592052]
399 [D loss: 0.651689, acc.: 62.50%] [G loss: 1.886492]
400 [D loss: 0.505088, acc.: 70.31%] [G loss: 1.800878]
401 [D loss: 0.515266, acc.: 71.88%] [G loss: 1.559229]
402 [D loss: 0.481422, acc.: 76.56%] [G loss: 2.025803]
403 [D loss: 0.560027, acc.: 67.19%] [G loss: 1.773435]
404 [D loss: 0.473397, acc.: 84.38%] [G loss: 1.722314]
405 [D loss: 0.457125, acc.: 73.44%] [G loss: 1.733802]
406 [D loss: 0.535415, acc.: 73.44%] [G loss: 1.669883]
407 [D loss: 0.542822, acc.: 73.44%] [G loss: 1.475660]
408 [D loss: 0.610837, acc.: 70.31%] [G loss: 1.681865]
409 [D loss: 0.516514, acc.: 68.75%] [G loss: 1.769084]
410 [D loss: 0.549807, acc.: 73.44%] [G loss: 1.

545 [D loss: 0.582801, acc.: 68.75%] [G loss: 1.348584]
546 [D loss: 0.561116, acc.: 68.75%] [G loss: 1.530760]
547 [D loss: 0.510911, acc.: 75.00%] [G loss: 1.333992]
548 [D loss: 0.581273, acc.: 68.75%] [G loss: 1.318477]
549 [D loss: 0.625420, acc.: 57.81%] [G loss: 1.257111]
550 [D loss: 0.706266, acc.: 54.69%] [G loss: 1.419835]
551 [D loss: 0.532407, acc.: 68.75%] [G loss: 1.348340]
552 [D loss: 0.573199, acc.: 71.88%] [G loss: 1.420793]
553 [D loss: 0.672983, acc.: 50.00%] [G loss: 1.241770]
554 [D loss: 0.550570, acc.: 70.31%] [G loss: 1.367924]
555 [D loss: 0.577981, acc.: 68.75%] [G loss: 1.126932]
556 [D loss: 0.611998, acc.: 65.62%] [G loss: 1.325098]
557 [D loss: 0.610961, acc.: 68.75%] [G loss: 1.331609]
558 [D loss: 0.579840, acc.: 75.00%] [G loss: 1.436043]
559 [D loss: 0.624678, acc.: 60.94%] [G loss: 1.452749]
560 [D loss: 0.593258, acc.: 67.19%] [G loss: 1.203843]
561 [D loss: 0.569563, acc.: 73.44%] [G loss: 1.488087]
562 [D loss: 0.539133, acc.: 71.88%] [G loss: 1.

...

19297 [D loss: 0.671315, acc.: 45.31%] [G loss: 0.778198]
19298 [D loss: 0.700736, acc.: 50.00%] [G loss: 0.785205]
19299 [D loss: 0.654115, acc.: 56.25%] [G loss: 0.773267]
19300 [D loss: 0.668514, acc.: 51.56%] [G loss: 0.773834]
19301 [D loss: 0.677821, acc.: 53.12%] [G loss: 0.830559]
19302 [D loss: 0.636585, acc.: 64.06%] [G loss: 0.803190]
19303 [D loss: 0.694129, acc.: 59.38%] [G loss: 0.725713]
19304 [D loss: 0.686848, acc.: 51.56%] [G loss: 0.728362]
19305 [D loss: 0.668947, acc.: 59.38%] [G loss: 0.809751]
19306 [D loss: 0.655328, acc.: 60.94%] [G loss: 0.730940]
19307 [D loss: 0.703761, acc.: 46.88%] [G loss: 0.762610]
19308 [D loss: 0.690534, acc.: 54.69%] [G loss: 0.767915]
19309 [D loss: 0.698327, acc.: 50.00%] [G loss: 0.793642]
19310 [D loss: 0.653597, acc.: 59.38%] [G loss: 0.830060]
19311 [D loss: 0.659236, acc.: 59.38%] [G loss: 0.767521]
19312 [D loss: 0.659644, acc.: 51.56%] [G loss: 0.799965]
19313 [D loss: 0.703686, acc.: 39.06%] [G loss: 0.793209]
19314 [D loss:

19443 [D loss: 0.649855, acc.: 67.19%] [G loss: 0.761393]
19444 [D loss: 0.621812, acc.: 62.50%] [G loss: 0.764089]
19445 [D loss: 0.650255, acc.: 62.50%] [G loss: 0.731156]
19446 [D loss: 0.668227, acc.: 60.94%] [G loss: 0.824578]
19447 [D loss: 0.637938, acc.: 57.81%] [G loss: 0.769270]
19448 [D loss: 0.670166, acc.: 53.12%] [G loss: 0.740423]
19449 [D loss: 0.626492, acc.: 60.94%] [G loss: 0.769538]
19450 [D loss: 0.655014, acc.: 54.69%] [G loss: 0.756227]
19451 [D loss: 0.631770, acc.: 59.38%] [G loss: 0.728155]
19452 [D loss: 0.730583, acc.: 50.00%] [G loss: 0.798564]
19453 [D loss: 0.736303, acc.: 50.00%] [G loss: 0.754864]
19454 [D loss: 0.670503, acc.: 59.38%] [G loss: 0.838750]
19455 [D loss: 0.674141, acc.: 57.81%] [G loss: 0.787592]
19456 [D loss: 0.668863, acc.: 60.94%] [G loss: 0.801908]
19457 [D loss: 0.658929, acc.: 64.06%] [G loss: 0.821000]
19458 [D loss: 0.665677, acc.: 62.50%] [G loss: 0.758452]
19459 [D loss: 0.676722, acc.: 56.25%] [G loss: 0.786814]
19460 [D loss:

19587 [D loss: 0.662470, acc.: 53.12%] [G loss: 0.820708]
19588 [D loss: 0.651726, acc.: 54.69%] [G loss: 0.770938]
19589 [D loss: 0.651462, acc.: 42.19%] [G loss: 0.784007]
19590 [D loss: 0.667409, acc.: 48.44%] [G loss: 0.758750]
19591 [D loss: 0.646257, acc.: 64.06%] [G loss: 0.724033]
19592 [D loss: 0.635271, acc.: 54.69%] [G loss: 0.800972]
19593 [D loss: 0.711927, acc.: 57.81%] [G loss: 0.773050]
19594 [D loss: 0.697573, acc.: 43.75%] [G loss: 0.803878]
19595 [D loss: 0.668496, acc.: 54.69%] [G loss: 0.746448]
19596 [D loss: 0.620220, acc.: 68.75%] [G loss: 0.841749]
19597 [D loss: 0.658824, acc.: 62.50%] [G loss: 0.868103]
19598 [D loss: 0.619602, acc.: 62.50%] [G loss: 0.764773]
19599 [D loss: 0.666062, acc.: 60.94%] [G loss: 0.832059]
19600 [D loss: 0.735374, acc.: 48.44%] [G loss: 0.739537]
19601 [D loss: 0.650520, acc.: 62.50%] [G loss: 0.846416]
19602 [D loss: 0.633857, acc.: 59.38%] [G loss: 0.821425]
19603 [D loss: 0.660673, acc.: 59.38%] [G loss: 0.835594]
19604 [D loss:

19734 [D loss: 0.675770, acc.: 46.88%] [G loss: 0.748532]
19735 [D loss: 0.631525, acc.: 65.62%] [G loss: 0.784446]
19736 [D loss: 0.705443, acc.: 43.75%] [G loss: 0.869121]
19737 [D loss: 0.704930, acc.: 53.12%] [G loss: 0.746294]
19738 [D loss: 0.632993, acc.: 64.06%] [G loss: 0.772235]
19739 [D loss: 0.648878, acc.: 62.50%] [G loss: 0.823420]
19740 [D loss: 0.634008, acc.: 70.31%] [G loss: 0.783546]
19741 [D loss: 0.690129, acc.: 59.38%] [G loss: 0.757875]
19742 [D loss: 0.688733, acc.: 46.88%] [G loss: 0.782601]
19743 [D loss: 0.666272, acc.: 54.69%] [G loss: 0.800664]
19744 [D loss: 0.642503, acc.: 56.25%] [G loss: 0.790404]
19745 [D loss: 0.640203, acc.: 60.94%] [G loss: 0.757536]
19746 [D loss: 0.677634, acc.: 46.88%] [G loss: 0.734344]
19747 [D loss: 0.734524, acc.: 50.00%] [G loss: 0.774211]
19748 [D loss: 0.641619, acc.: 68.75%] [G loss: 0.789730]
19749 [D loss: 0.704647, acc.: 50.00%] [G loss: 0.772129]
19750 [D loss: 0.703620, acc.: 51.56%] [G loss: 0.774806]
19751 [D loss:

19881 [D loss: 0.683701, acc.: 59.38%] [G loss: 0.792400]
19882 [D loss: 0.709525, acc.: 53.12%] [G loss: 0.738026]
19883 [D loss: 0.685486, acc.: 64.06%] [G loss: 0.804663]
19884 [D loss: 0.669639, acc.: 50.00%] [G loss: 0.764870]
19885 [D loss: 0.657894, acc.: 64.06%] [G loss: 0.804942]
19886 [D loss: 0.688924, acc.: 51.56%] [G loss: 0.800648]
19887 [D loss: 0.680893, acc.: 45.31%] [G loss: 0.790039]
19888 [D loss: 0.672629, acc.: 60.94%] [G loss: 0.753449]
19889 [D loss: 0.658116, acc.: 71.88%] [G loss: 0.765726]
19890 [D loss: 0.623975, acc.: 59.38%] [G loss: 0.799227]
19891 [D loss: 0.689968, acc.: 46.88%] [G loss: 0.718268]
19892 [D loss: 0.669164, acc.: 57.81%] [G loss: 0.751528]
19893 [D loss: 0.683920, acc.: 48.44%] [G loss: 0.744191]
19894 [D loss: 0.752595, acc.: 42.19%] [G loss: 0.791314]
19895 [D loss: 0.705575, acc.: 40.62%] [G loss: 0.758384]
19896 [D loss: 0.703238, acc.: 54.69%] [G loss: 0.809251]
19897 [D loss: 0.649170, acc.: 59.38%] [G loss: 0.778000]
19898 [D loss: